In [1]:
%pylab inline
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd
import gzip
import pickle
import random
import seaborn 


Populating the interactive namespace from numpy and matplotlib


### Parsing the pfam proteome annnotation into a graph 

`generate_graph()` takes a pfam proteome file and parses it into a networkx graph. 

It parses the following features for **nodes** : 
  * seqid (PFAM accession number)
  * type can be *protein* or one of domain, family clan etc. from pfam
  * name (only for non-protein types)

It parses the following features for **edges** : 
  * bitscore
  * e-value

In [2]:
def generate_graph(accession):
    g = nx.Graph()
    proteins=set()
    domains=set()
    
    with gzip.open('../data/pfam/proteomes/' + accession + '.tsv.gz', mode='rt') as f : 
        for i,line in enumerate(f) : 
            if line[0] == '#' :
                continue
            else :
                line = line.strip().split()
                #print (line[0], line[5])
            #    if line[7] =='Domain' :
                seqid = line[0]
                hmmid = line[5]
                name = line[6]
                hmm_type = line[7]
                bitscore = float(line[11])
                evalue = float(line[12])
                clan = line[13]
                
                g.add_edge(seqid, hmmid)
                g[seqid][hmmid]['bitscore'] = bitscore
                g[seqid][hmmid]['evalue'] = evalue
                proteins.add(seqid)
                domains.add(hmmid)
                g.node[seqid]['type']='protein'
                g.node[hmmid]['type'] = hmm_type
                g.node[hmmid]['name'] = name
    
    return g, domains, proteins

# little helper function to find the component containing a given domain

def component_containing(h, node) :
    for component in nx.connected_component_subgraphs(h) :
        if component.has_node(node) :
            return component


We use organisms that have >80% coverage and more than 1000 proteins in the proteome. 
These are saved from another notebook titled **parse_pfam_index**. 
Some relevant stats are given there.

In [3]:
organisms = pickle.load(open('list.pkl', 'rb'))
print ('there are {} organisms in the set'.format(len(organisms)))

there are 2266 organisms in the set


These are the c-d-GMP **binding** domains. 
There are many more that are c-d-GMP **associated**, but these are those with experimental evidence (says Melene)
  
  * PilZ  [PF07238](http://pfam.xfam.org/family/PF07238)
  * GGDEF [PF00990](http://pfam.xfam.org/family/ggdef)
  * GIL [PF10995](http://pfam.xfam.org/family/PF10995)
  * EAL [PF00563](http://pfam.xfam.org/family/PF00563)
  * HD_5, PF13487
  * FleQ, PF06490
  * MshEN, PF05157


  * PilZ_2 [PF16823](http://pfam.xfam.org/family/PF16823)

In [23]:
relevant_domains = {
    'PF07238':'PilZ', 
    'PF00990':'GGDEF', 
    'PF00563':'EAL',
    'PF10995':'GIL',
    'PF13487':'HD_5',
    'PF06490':'FleQ',
    'PF05157':'MshEN'
}

### Extracting features

This is where we extract a number of network based features for each organism.
The networks are loaded from precomputed files if they exist, or calculated from PFAM proteome annotations using `generate_graph()` described above. 
The features we are extracting **from each proteome**: 
  * **nprotein** number of proteins
  * **ndomain** number of domains
  * **ncomps** number of connected components 
  * **majorcomp** size of the largest connected component
  * for each `relevant_domain` (described above)
    * **domain** existence of this domain (True/False)
    * **domain_k** degree of this domain 
    * **domain_c** clustering coefficient of this domain
    * **domain_n** size of the component containing this domain
    * **domain_bc** betweennes centrality of this domain
    * **domain_ec** eigenvector centrality of this domain

In [127]:
species_features=dict()
#species_graphs=dict()

n=100

#for acc in random.sample(list(organisms), n) : 
for acc in list(organisms) : 
    
    try : 
        # trying to read the graph from a file
        g = nx.read_gml(acc + '_bipartite.gml')
        
        domains = set()
        proteins = set()
        
        for n in g.nodes_iter() :
            if g.node[n]['type'] == 'protein' :
                proteins.add(n)
            else : 
                domains.add(n)
        
    except :
        # parsing the graph from pfam file
        g, domains, proteins = generate_graph(acc)
    
        # saving the graph for later use
        nx.write_gml(g, acc + '_bipartite.gml')
    
    this=dict() #feature dict
    
    # basic features
    this['nproteins'] = len(proteins)
    this['ndomains'] = len(domains)
    
    # projecting the bipartite graph onto domains
    h = bipartite.projected_graph(g,domains)

    #nx.write_gml(h, acc + '_domains.gml')
    
    # component features
    
    this['ncomps']    = nx.number_connected_components(h)
    this['majorcomp'] = len(max(nx.connected_components(h), key=len))
    
    for domain in relevant_domains : 
        dname = relevant_domains[domain]
        exists = domain in h 
        
        if exists : 
            this[dname] = True
            degree = nx.degree(h,domain)
            this[dname+'_k'] = degree
            
            if degree>0 :
            
                this[dname+'_c'] = nx.clustering(h, domain)
            
                component = component_containing(h,domain)
                this[dname+'_n']= len(component)

                
                if degree >1 : 
                    this[dname+'_bc'] = nx.betweenness_centrality(component)[domain]
                    this[dname+'_ec'] = nx.eigenvector_centrality_numpy(component)[domain]
                    
                
            else : 
                this[dname+'_n']= 1
            
        else :
            this[dname] = False

    species_features[organisms[acc]]= this
   # species_graphs[organisms[acc]]= h

# convert into a dataframe, pickle and print description
data=pd.DataFrame.from_dict(species_features, orient='index')
f=open('data.pkl', 'wb')
pickle.dump(data, f)
data.describe()

,nproteins,ndomains,ncomps,majorcomp,PilZ_k,PilZ_c,PilZ_n,PilZ_bc,PilZ_ec,GGDEF_k,...,MshEN_n,MshEN_bc,MshEN_ec,FleQ_k,FleQ_c,FleQ_n,FleQ_bc,FleQ_ec,GIL_k,GIL_n
count,2266.000000,2266.000000,2266.000000,2266.000000,1229.000000,856.000000,1229.000000,341.000000,3.410000e+02,1751.000000,...,1113.000000,149.000000,1.490000e+02,203.000000,202.000000,203.00000,192.0,192.000000,74.0,74.0
mean,2816.634157,1818.064431,1165.292586,74.719329,1.167616,0.106724,16.133442,0.421568,3.518067e-01,8.960594,...,11.803235,0.362275,3.306390e-01,1.940887,0.950495,109.26601,0.0,0.015009,0.0,1.0
std,1141.722047,410.401002,260.825043,52.311773,1.181932,0.266607,41.898455,0.402929,2.946933e-01,6.709579,...,39.599568,0.469166,3.247544e-01,0.256506,0.217459,40.62675,0.0,0.024782,0.0,0.0
min,823.000000,697.000000,463.000000,8.000000,0.000000,0.000000,1.000000,0.000000,1.951779e-12,0.000000,...,1.000000,0.000000,6.331922e-07,0.000000,0.000000,1.00000,0.0,0.003978,0.0,1.0
25%,1914.500000,1499.250000,960.000000,39.000000,0.000000,0.000000,1.000000,0.022989,4.167184e-02,3.000000,...,2.000000,0.000000,1.494365e-02,2.000000,1.000000,78.00000,0.0,0.009430,0.0,1.0
50%,2702.000000,1840.000000,1168.000000,57.000000,1.000000,0.000000,2.000000,0.285714,3.424853e-01,8.000000,...,2.000000,0.000000,1.436266e-01,2.000000,1.000000,111.00000,0.0,0.011736,0.0,1.0
75%,3513.500000,2127.750000,1373.000000,96.000000,2.000000,0.000000,4.000000,0.833333,6.532815e-01,13.000000,...,2.000000,1.000000,7.071068e-01,2.000000,1.000000,138.00000,0.0,0.015391,0.0,1.0
max,12055.000000,3204.000000,2010.000000,341.000000,10.000000,1.000000,335.000000,1.000000,7.071068e-01,36.000000,...,335.000000,1.000000,7.071068e-01,2.000000,1.000000,224.00000,0.0,0.342672,0.0,1.0
